In [1]:
from functools import wraps
from functools import reduce

from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
import cv2             # working with, mainly resizing, images
from tqdm import tqdm
from lxml import etree
import xml.etree.ElementTree as ET
import random
import os                  # dealing with directories
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D, Input, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras import models
from keras import layers
from keras.layers import Input

#TO DO
#z loss function vyjebat classes
#scalenut poslednu layer na moj output
#k means clustering = getting nice anchors https://machinethink.net/blog/object-detection/
#reading input

Using TensorFlow backend.


In [2]:
TRAIN_dir = 'D:/cnntry/air_data/trainairplane'
TRAIN_annot_dir = 'D:/cnntry/air_data/annot_train'
img_folder='D:/cnntry/air_data/trainairplane'

#TEST_DIR = 'D:\cnntry\\test'
TEST_DIR ='andrej/datasets/cats-and-dogs/1/home/test'

DATA_file= 'train_airdata.npy' 


def get_label(img):
    annot_label = img.split('.')[0]+'.xml'           #xml file name
    annot_xml = os.path.join(TRAIN_annot_dir,annot_label)  #xml file full path
    
    root = etree.parse(annot_xml).getroot()
    #for child in root: print(child.tag, child.attrib)
    count = sum(1 for _ in root.iter("object")) #count of objects in xml file
    a = random.getrandbits(1)
    if count == 1 : return [1,0]  # if only 1 airplane
    else: return [0,1]            # of more than one airplane

In [52]:
anchors='10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326'
anchors = np.array([float(x) for x in anchors.split(',')]).reshape(-1, 2)

model = models.Sequential()
input_shape = (416,416)
h, w = input_shape
num_anchors = len(anchors)
num_classes = 1
y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(1)]

model.add(layers.Conv2D(16, 3, strides=(1, 1), padding='same', kernel_regularizer= l2(5e-4),input_shape=(416, 416, 3)))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=0.1))


filters=[32,64,128,256,512]#,1024]
for fil in filters:
    model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(layers.Conv2D(fil, 3, strides=(1, 1), padding='same', kernel_regularizer= l2(5e-4))) 
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.1))


model.add(layers.Conv2D(512, 3, strides=(1, 1), padding='same', kernel_regularizer= l2(5e-4)))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Conv2D(5, 3, strides=(1, 1), padding='same', activation='linear', kernel_regularizer= l2(5e-4)))  

model.compile('SGD', loss='mean_squared_error')#,target_tensor=(200, 13, 13, 5))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 416, 416, 16)      448       
_________________________________________________________________
batch_normalization_36 (Batc (None, 416, 416, 16)      64        
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 416, 416, 16)      0         
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 208, 208, 16)      0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 208, 208, 32)      4640      
_________________________________________________________________
batch_normalization_37 (Batc (None, 208, 208, 32)      128       
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 208, 208, 32)      0         
__________

In [ ]:
#model.fit(train_images, train_labels, epochs=5, batch_size=64)

#test_loss, test_accuracy = model.evaluate(validation_images, validation_labels)

#print(test_accuracy)

In [53]:
def read_xml(img_name):
    annot_label = img_name.split('.')[0]+'.xml'           #xml file name
    annot_xml = os.path.join(TRAIN_annot_dir,annot_label)  #xml file full path
    root = etree.parse(annot_xml).getroot()
    #for child in root: print(child.tag, child.attrib)        
    img_w= int(root[3][0].text)
    img_h= int(root[3][1].text)
    bb=[]  #list for bounding boxes
    for plane in root.iter("object"):
        for l in range(4): bb.append(int(plane[4][l].text)) 
    return img_w, img_h, bb

def rescale_bb(img_w, img_h, bb, ref=(416,416)):
    w_ratio= ref[0] / img_w
    h_ratio= ref[1] / img_h
    for i in range(0,len(bb),2):
        bb[i]= int(bb[i]* w_ratio)
        bb[i+1]= int(bb[i+1]* h_ratio)
    return bb

def conv_bb_coord(bb):
    for i in range(0,len(bb),4):
        x_cen= int( (bb[i]+ bb[i+2])/2)
        y_cen= int( (bb[i+1]+ bb[i+3])/2)
        w= bb[i+2]- bb[i]
        h= bb[i+3]- bb[i+1]
        bb[i],bb[i+1],bb[i+2],bb[i+3]= x_cen, y_cen, w, h
    return bb


def create_truth_arr(bb, output_shape=(13,13,5), ref=(416,416)):    #bb[x_cen, y_cen, w, h, x_cen, y_cen, w, h..]
    truth=np.zeros((output_shape))
    box_w= ref[0]/output_shape[0]
    for i in range(0,len(bb),4):
        for j in range(output_shape[0]):
            if bb[i]>= j*box_w  and  bb[i]< (j+1)*box_w:
                a=j
            if bb[i+1]>= j*box_w  and  bb[i+1]< (j+1)*box_w:
                b=j
                
        truth[a,b,0:]= 1,*bb[i:i+4]
        truth[a,b,1:]= truth[a,b,1:]/ref[1]
    return truth
    

def prep_pics(img_folder='D:/cnntry/air_data/trainairplane',ref=(416,416)):
    training_data = []
    x_data=np.zeros((200,416, 416, 3))
    y_data=np.zeros((200,13,13,5))
    n=0
    for img in tqdm(os.listdir(img_folder)):
        n=n+1
        if n>200:
            break
        truth= create_truth_arr(conv_bb_coord(rescale_bb(*read_xml(img))))
        path = os.path.join(img_folder,img)
        img = cv2.imread(path)
        img = cv2.resize(img, ref)
        x_data[n-1,...]=np.array(img)
        y_data[n-1,...]=truth
        #training_data.append([np.array(img),truth])
    #shuffle(training_data)
    np.save('x_data.npy', x_data)#training_data) 
    np.save('y_data.npy', y_data)
    return x_data, y_data #,training_data
    
    
    
print(read_xml('000000000913.jpg'))
print(rescale_bb(*read_xml('000000000913.jpg')))
print(conv_bb_coord(rescale_bb(*read_xml('000000000913.jpg'))))
#print(create_truth_arr(conv_bb_coord(rescale_bb(*read_xml('000000000913.jpg')))))

x_data, y_data=prep_pics()

(640, 512, [50, 126, 636, 282, 0, 161, 208, 239])
[32, 102, 413, 229, 0, 130, 135, 194]
[222, 165, 381, 127, 67, 162, 135, 64]


  7%|█████▏                                                                         | 198/2986 [00:04<00:50, 55.45it/s]


In [54]:
model.fit(x=x_data, y=y_data, batch_size=50, epochs=1, verbose=1, validation_split=0.2, shuffle=True)

Train on 160 samples, validate on 40 samples
Epoch 1/1
160/160 [==============================] - ETA: 1:31 - loss: 1.574 - ETA: 43s - loss: 2.105 - ETA: 7s - loss: 4.1780 - 123s 770ms/step - loss: 4.5741 - val_loss: 4.1500


In [46]:
y_data.shape


(200, 13, 13, 5)